# INVENTA - Prioritization of natural extracts for chemical originality discovery


In [1]:
import sys 
sys.path.append('../src')

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import plotly.express as px
import zipfile

import lineup_widget
from __future__ import print_function
from ipywidgets import interact, interactive, interact_manual
!jupyter nbextension enable --py --sys-prefix lineup_widget

from sklearn.metrics import pairwise_distances
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
from skbio.stats.ordination import pcoa
from skbio import OrdinationResults

from inventa import quant_table
from inventa import reduce_df
from inventa import full_data
from inventa import top_ions
from inventa import annotations
from inventa import feature_component
from inventa import literature_component
from inventa import similarity_component
from inventa import sirius_classes
from inventa import search_reported_class
from inventa import class_component
from inventa import process_gnps_results
from inventa import drop_samples_based_on_string
from inventa import process_gnps_results

from plot import pcoa_2d

Enabling notebook extension lineup_widget/extension...
      - Validating: OK


![title](../src/Priority_rank.png)

### Adapt the filenames accordig to your data

In [2]:
#Input filenames: drag them in the data folder

metadata_filename = '../data/Celastraceae_Set_metadata_pos.tsv'
quantitative_data_filename = '../data/Celastraceae_pos_quant.csv'
tima_results_filename = '../data/Celastraceae_pos_spectral_match_results_repond.tsv'
vectorized_data_filename = '../data/Celastraceae_memomatrix.csv'
sirius_results_filename = '../data/canopus_npc_summary.tsv'

### Download GNPS results

In [3]:
!del all_annotations.zip

# Replace the job ID in the line below <<< ====
!curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=4c919fcbc83d487493a487012afb920a&view=view_all_annotations_DB" -o ../data/all_annotations.zip
!pwd
!ls ../data
!unzip -q -d ../data/all_annotations/ -o ../data/all_annotations.zip

print('==================')
print('GNPS job results were succesfully downloaded')

clusterinfosummary = process_gnps_results('../data/all_annotations')

/bin/bash: del: command not found
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.9M    0 38.9M    0     0  3534k      0 --:--:--  0:00:11 --:--:-- 5396k12.9M    0     0  2083k      0 --:--:--  0:00:06 --:--:-- 2644k
/mnt/c/Users/quirosgu/Documents/GitHub/INVENTA/notebook
198a574e172443ad84f43e739ceea2c8.tsv
Celastraceae_Set_metadata_pos.tsv
Celastraceae_memomatrix.csv
Celastraceae_pos_quant.csv
Celastraceae_pos_spectral_match_results_repond.tsv
all_annotations
all_annotations.zip
canopus_npc_summary.tsv
GNPS job results were succesfully downloaded
FBMN job detected
   Number of spectral library annotations in job = 1836
   Number of network nodes in the job = 14970


## Prepare input files

### Metadata table

In [4]:
#metadata 
metadata_df = pd.read_csv(metadata_filename, sep='\t')

#if a column Species\part is NOT present run the following command, if IS PRESENT don't run.
metadata_df['ATTRIBUTE_Sppart'] = metadata_df["ATTRIBUTE_Species"]+ "|" + metadata_df["ATTRIBUTE_Organe"].map(str)
metadata_df.head(3)

,filename,ATTRIBUTE_Code,ATTRIBUTE_Type,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,ATTRIBUTE_Organe,ATTRIBUTE_Broad_organ,ATTRIBUTE_Tissue,ATTRIBUTE_Subsystem,ATTRIBUTE_Sppart
0,LQ-01-61-01_pos.mzXML,V107694,Sample,Celastraceae,Catha,Catha edulis,Leaves,photosynthetic,green tissue,aboveground,Catha edulis|Leaves
1,LQ-01-61-02_pos.mzXML,V107695,Sample,Celastraceae,Catha,Catha edulis,Stems,woody vegetative,woody tissue,aboveground,Catha edulis|Stems
2,LQ-01-61-03_pos.mzXML,V107696,Sample,Celastraceae,Catha,Catha edulis,Roots,roots,root tissue,belowground,Catha edulis|Roots


### Quantification table

In [5]:
#Quantitative table
quant_df = pd.read_csv(quantitative_data_filename, sep=',', index_col='row ID')
quant_df = quant_table(quant_df)
quant_df.head(3)

,LQ-01-61-01_pos.mzXML,LQ-01-61-02_pos.mzXML,LQ-01-61-03_pos.mzXML,LQ-01-61-04_pos.mzXML,LQ-01-61-05_pos.mzXML,LQ-01-61-06_pos.mzXML,LQ-01-61-07_pos.mzXML,LQ-01-61-08_pos.mzXML,LQ-01-61-09_pos.mzXML,LQ-01-61-10_pos.mzXML,...,LQ-01-61-69_pos.mzXML,LQ-01-61-70_pos.mzXML,LQ-01-61-71_pos.mzXML,LQ-01-61-72_pos.mzXML,LQ-01-61-73_pos.mzXML,LQ-01-61-74_pos.mzXML,LQ-01-61-75_pos.mzXML,LQ-01-61-76_pos.mzXML,LQ-01-61-77_pos.mzXML,LQ-01-61-78_pos.mzXML
row ID,,,,,,,,,,,,,,,,,,,,,
1,2.600773e+09,3.134767e+07,8.413720e+07,6.925458e+07,0.0,0.0,0.0,0.0,3.256476e+06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.860196e+08,1.964781e+07,8.688867e+06,2.214498e+09,0.0,0.0,0.0,0.0,1.639854e+06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.544583e+08,2.594163e+07,0.000000e+00,4.612163e+08,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Merge tables

In [6]:
full_df = full_data(metadata_df,quant_df)
print(full_df.shape)
full_df.head(2)

(78, 14981)


,filename,ATTRIBUTE_Code,ATTRIBUTE_Type,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,ATTRIBUTE_Organe,ATTRIBUTE_Broad_organ,ATTRIBUTE_Tissue,ATTRIBUTE_Subsystem,...,14961,14962,14963,14964,14965,14966,14967,14968,14969,14970
0,LQ-01-61-01_pos.mzXML,V107694,Sample,Celastraceae,Catha,Catha edulis,Leaves,photosynthetic,green tissue,aboveground,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LQ-01-61-02_pos.mzXML,V107695,Sample,Celastraceae,Catha,Catha edulis,Stems,woody vegetative,woody tissue,aboveground,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Remove experimental controls

In [7]:
#erase all the blanks and QC's - Change the string as needed
list_of_strings_for_QC_Blank_filter = ['Blank', 'QC']
column_to_use_for_filtering = 'ATTRIBUTE_Type' #this information should be included in the metadata table

full_df = drop_samples_based_on_string(full_df,list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)
metadata_df = drop_samples_based_on_string(metadata_df,list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)

(78, 14981)
(76, 14981)
(77, 11)
(76, 11)


In [8]:
# Make a minimal table for further processing
reduced_df = reduce_df(full_df, metadata_df, 'ATTRIBUTE_Sppart')
reduced_df.head(2)

,1,2,3,4,5,6,7,8,9,10,...,14961,14962,14963,14964,14965,14966,14967,14968,14969,14970
ATTRIBUTE_Sppart,,,,,,,,,,,,,,,,,,,,,
Catha edulis|Leaves,2.600773e+09,8.860196e+08,8.544583e+08,6.320161e+08,5.282800e+08,5.227184e+08,5.122110e+08,3.936131e+08,4.722463e+08,2.347275e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Catha edulis|Stems,3.134767e+07,1.964781e+07,2.594163e+07,0.000000e+00,7.494122e+06,2.107071e+08,0.000000e+00,5.018283e+07,1.859377e+07,3.222927e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature component (FC)

### FC. Feature Specifificty

In [9]:
specificity_df = top_ions(reduced_df, quant_df)
specificity_df.head(2)

,row ID,filename,ATTRIBUTE_Sppart,Feature_specificity
0,1,LQ-01-61-01_pos.mzXML,Catha edulis|Leaves,93.258820
1,2,LQ-01-61-04_pos.mzXML,Catha edulis|Aerial_parts,69.662011


### FC. Annotation Rate

In [10]:
annot_gnps_df = pd.read_csv(clusterinfosummary, 
                                sep='\t', 
                                usecols=['cluster index','componentindex', 'SpectrumID'], 
                                low_memory=False)
annot_is_df = pd.read_csv(tima_results_filename,
                                sep='\t', 
                                usecols =['libname','feature_id','short_inchikey'], 
                                low_memory=False)

annotation_df = annotations(annot_gnps_df, annot_is_df)
annotation_df.head(2)

,cluster index,componentindex,Annotated_GNPS,annotation
0,1,113,0,0
1,2,136,0,0


In [11]:
FC = feature_component(specificity_df, annotation_df, metadata_df)
#FC = FC.sort_values(by=['FC'], ascending=False)
FC.head(2)

,filename,ATTRIBUTE_Species,ATTRIBUTE_Sppart,Sample_specificity,FC
5,LQ-01-61-06_pos.mzXML,Celastrus orbiculatus,Celastrus orbiculatus|Roots,0.863281,0.824219
75,LQ-01-61-78_pos.mzXML,Pristimera indica,Pristimera indica|Roots,0.841667,0.813889


# Literature component (LC)


In [12]:
#erase all the blanks and QC's

LC = literature_component(metadata_df)
LC = LC.sort_values(by=['LC'], ascending=False)
LC.head(2)

,filename,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,Reported_comp_Family,Reported_comp_Genus,Reported_comp_Species,LC
43,LQ-01-61-40_pos.mzXML,Celastraceae,Loeseneriella,Loeseneriella urceolus,0.0,0.0,0,1
50,LQ-01-61-47_pos.mzXML,Celastraceae,Maytenus,Maytenus inermis,0.0,0.0,0,1


# Similarity component (SC)

In [13]:
metric_df = pd.read_csv(vectorized_data_filename, sep=',', encoding= 'unicode_escape')
metric_df.head(3)

,filename,peak@53.04,peak@57.07,peak@77.04,peak@78.03,peak@83.09,peak@85.07,peak@91.05,peak@93.07,peak@95.05,...,peak@470.31,peak@185.87,peak@286.39,peak@655.05,peak@158.75,peak@386.73,peak@411.57,peak@611.62,peak@466.50,peak@123.63
0,LQ-01-61-01_pos.mzXML,45.0,246.0,23.0,66.0,239.0,147.0,230.0,293.0,191.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LQ-01-61-02_pos.mzXML,32.0,290.0,8.0,16.0,338.0,159.0,285.0,395.0,144.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LQ-01-61-03_pos.mzXML,31.0,259.0,8.0,35.0,317.0,131.0,301.0,468.0,131.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
SC = similarity_component(metric_df[:-1])
SC.head(2)

/home/quirosgu/anaconda3/envs/inventa/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,filename,anomaly_IF,anomaly_LOF,anomaly_OCSVM,SC
0,LQ-01-61-01_pos.mzXML,1,1,1,0
1,LQ-01-61-02_pos.mzXML,1,1,1,0


#### Visualizing the similarity results

In [19]:
# Isolation forest (IF)
trace1 = pcoa_2d(
    matrix= metric_df[:-1],
    data = SC,
    metric= 'braycurtis',
    filename_col = 'filename',
    group_col='anomaly_IF',
    title='"Isolation Forest & PCoA (bray-curtis) based on the feature matrix"'
    )

In [17]:
# Local outlier factor (LOF)
trace2 = pcoa_2d(
    matrix= metric_df[:-1],
    data = SC,
    metric= 'braycurtis',
    filename_col = 'filename',
    group_col='anomaly_LOF',
    title='"Local outlier factor & PCoA (bray-curtis) based on the feature matrix"'
    )

In [18]:
# One-class support vector machine (OCSVM)
trace3 = pcoa_2d(
    matrix= metric_df[:-1],
    data = SC,
    metric= 'braycurtis',
    filename_col = 'filename',
    group_col='anomaly_OCSVM',
    title='"One-class support vector machine & PCoA (bray-curtis) based on the feature matrix"'
    )

## Chemical classes from Sirius

### CC. Chemical class

In [15]:
#Upload sirius results 
sirius_df            = pd.read_csv(sirius_results_filename ,
                                sep='\t').dropna()
#SIRIUS.head(2)

sirus_classes_df = sirius_classes(specificity_df, metadata_df,sirius_df)
sirus_classes_df.head(2)

,filename,classe
0,LQ-01-61-01_pos.mzXML,"{Pyridine alkaloids, Aminoacids, Piperidine al..."
1,LQ-01-61-02_pos.mzXML,"{Pyridine alkaloids, Cholane steroids, Labdane..."


### CC. Search for reported chemical classes according to the species in the set

In [16]:
reported_classes_df = search_reported_class(metadata_df)
reported_classes_df.head(2)

,filename,ATTRIBUTE_Species,ATTRIBUTE_Genus,ATTRIBUTE_Family,ATTRIBUTE_Family,ATTRIBUTE_Sppart,Chemical_class_reported_in_species,Chemical_class_reported_in_genus
0,LQ-01-61-01_pos.mzXML,Catha edulis,Catha,Celastraceae,Celastraceae,Catha edulis|Leaves,"{Pyridine alkaloids, Aminoacids, Thujane monot...","{Pyridine alkaloids, Aminoacids, Thujane monot..."
1,LQ-01-61-02_pos.mzXML,Catha edulis,Catha,Celastraceae,Celastraceae,Catha edulis|Stems,"{Pyridine alkaloids, Aminoacids, Thujane monot...","{Pyridine alkaloids, Aminoacids, Thujane monot..."


### CC. Computation of CC

In [17]:
CC = class_component(reported_classes_df, sirus_classes_df)
CC.head(2)

,filename,ATTRIBUTE_Species,ATTRIBUTE_Genus,ATTRIBUTE_Family,ATTRIBUTE_Family,ATTRIBUTE_Sppart,Chemical_class_reported_in_species,Chemical_class_reported_in_genus,classe,New_in_species,New_in_genus,CC
0,LQ-01-61-01_pos.mzXML,Catha edulis,Catha,Celastraceae,Celastraceae,Catha edulis|Leaves,"{Pyridine alkaloids, Aminoacids, Thujane monot...","{Pyridine alkaloids, Aminoacids, Thujane monot...","{Pyridine alkaloids, Aminoacids, Piperidine al...","{Caryophyllane sesquiterpenoids, Polyether ion...","{Caryophyllane sesquiterpenoids, Polyether ion...",1
1,LQ-01-61-02_pos.mzXML,Catha edulis,Catha,Celastraceae,Celastraceae,Catha edulis|Stems,"{Pyridine alkaloids, Aminoacids, Thujane monot...","{Pyridine alkaloids, Aminoacids, Thujane monot...","{Pyridine alkaloids, Cholane steroids, Labdane...","{Cholane steroids, Androstane steroids, Labdan...","{Cholane steroids, Androstane steroids, Labdan...",1


# Priority rank Results

In [18]:
PR =pd.merge(
                left=FC,
                right=LC[['filename', 'Reported_comp_Species', 'Reported_comp_Genus', 'LC', 'ATTRIBUTE_Family']], 
                how='left', 
                left_on='filename', 
                right_on='filename')
PR =pd.merge(
                    left=PR,
                    right=SC[['filename', 'SC']], 
                    how='left', 
                    left_on='filename', 
                    right_on='filename')
PR =pd.merge(
                    left=PR,
                    right=CC[['filename', 'New_in_species', 'New_in_genus', 'CC']], 
                    how='left', 
                    left_on='filename', 
                    right_on='filename')
PR['CC'] = PR['CC'].fillna(0)

In [19]:
#specify the weight to modulate each component 
w1 = 1
w2 = 1
w3 = 1
w4 = 1

PR['PR'] = w1*PR['FC'] + w2*PR['LC'] + w3*PR['SC'] + w4*PR['CC']
PR = PR.sort_values(by=['PR'], ascending=False)
PR.head()

,filename,ATTRIBUTE_Species,ATTRIBUTE_Sppart,Sample_specificity,FC,Reported_comp_Species,Reported_comp_Genus,LC,ATTRIBUTE_Family,SC,New_in_species,New_in_genus,CC,PR
7,LQ-01-61-37_pos.mzXML,Euonymus sanguineus,Euonymus sanguineus|Roots,0.838532,0.739450,1,515.0,1,Celastraceae,1.0,"{Phenethylisoquinoline alkaloids, Aminoacids, ...","{Phenethylisoquinoline alkaloids, Aminoacids, ...",1.0,3.739450
15,LQ-01-61-07_pos.mzXML,Celastrus paniculatus,Celastrus paniculatus|Seeds,0.713450,0.670565,2,910.0,1,Celastraceae,1.0,"{Polyamines, Diacylglycerols, Prenyl quinone m...","{Polyamines, Diacylglycerols, Prenyl quinone m...",1.0,3.670565
0,LQ-01-61-06_pos.mzXML,Celastrus orbiculatus,Celastrus orbiculatus|Roots,0.863281,0.824219,253,910.0,0,Celastraceae,1.0,"{Aminoacids, Polyamines, Phenylalanine-derived...","{Aminoacids, Polyamines, Phenylalanine-derived...",1.0,2.824219
4,LQ-01-61-27_pos.mzXML,Euonymus fortunei,Euonymus fortunei|Aerial_parts,0.903226,0.790323,1,515.0,1,Celastraceae,0.0,"{Flavanones, Phenylalanine-derived alkaloids, ...","{Phenylalanine-derived alkaloids, Piperidine a...",1.0,2.790323
6,LQ-01-61-23_pos.mzXML,Euonymus cochinchinensis,Euonymus cochinchinensis|Leaves,0.805031,0.764151,0,0.0,1,Celastraceae,1.0,NaN,NaN,0.0,2.764151


### Display results

In [21]:
#Show the results in an interactive way
def selection_changed(selection):
    return PR.iloc[selection]
interact(selection_changed, selection=lineup_widget.LineUpWidget(PR));

interactive(children=(LineUpWidget(value=[], description='selection', layout=Layout(align_self='stretch', heig…

In [22]:
#Save it as .csv 
PR.to_csv('../results/INVENTA_results.tsv', sep='\t')